## [CamemBERTsum](https://huggingface.co/mrm8488/camembert2camembert_shared-finetuned-french-summarization) fine-tuned on [mlsum-fr](https://huggingface.co/datasets/viewer/?dataset=mlsum)

> MLSUM is the first large-scale MultiLingual SUMmarization dataset. Obtained from **online newspapers**, it contains **1.5M+ article/summary pairs** in five different languages -- namely, **French**, German, Spanish, Russian, Turkish.
> https://huggingface.co/mrm8488/camembert2camembert_shared-finetuned-french-summarization#dataset

>* Size of downloaded dataset files: 591.27 MB
>* Size of the generated dataset: 1537.36 MB
>* Total amount of disk used: 2128.63 MB
>* An example of 'validation' looks as follows.
>```json
{
    "date": "01/01/2001",
    "summary": "A text",
    "text": "This is a text",
    "title": "A sample",
    "topic": "football",
    "url": "https://www.google.com"
}
>```
>https://huggingface.co/datasets/mlsum#fr

In [1]:
from pathlib import Path
from pprint import PrettyPrinter

import torch
from transformers import RobertaTokenizerFast, EncoderDecoderModel

from book_loader import BookLoader

In [2]:
data_path = Path("data/D5627-Dolan.docx").expanduser().resolve()
book = BookLoader(data_path=data_path)
chapters: list[list[str]] = book.chapters

/home/ahmedm/anaconda3/envs/LDS/lib/python3.10/site-packages/simplify_docx/iterators/generic.py:193: UnexpectedElementWarning: Skipping unexpected tag: {http://schemas.openxmlformats.org/wordprocessingml/2006/main}pict
  warn("Skipping unexpected tag: %s" % (current.tag),


In [3]:
torch.cuda.is_available()
# TODO: Investigate this. Maybe has to do with exe install task you need Samuel for.
#       Other solution is to run this directly in WSL as a .py

False

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
ckpt = 'mrm8488/camembert2camembert_shared-finetuned-french-summarization'
tokenizer = RobertaTokenizerFast.from_pretrained(ckpt)
model = EncoderDecoderModel.from_pretrained(ckpt).to(device)

The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']


In [5]:
def generate_summary(text):
    inputs = tokenizer([text], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    output = model.generate(input_ids, attention_mask=attention_mask)
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [6]:
# TODO: 1- Take out paragraphs starting with "^exerCiCe \d.\d /$" and ending with chapter_marker
#        - Verify consistency of chapter_marker as an end bound
# TODO: 2- Take out tables before summarizing, or figure out a way to extract useful info from them
# TODO: 3- In the docx earch for: "NOTE: LE BLOC CI-DESSUS N’EST PAS CAPTURÉ"

In [7]:
# TODO: Patching out tables on-the-fly until it's done in book_loaders.py
chapters = ['\n'.join(paragraph for paragraph in chapter if isinstance(paragraph, str))
            for chapter in book.chapters]
print(chapters[1])

Stress, santé et travail: vue d'ensemble
J'oublie ce que j'entends, je me rappelle ce que je vois, je comprends ce que je fais.
CONFUCIUS
/	Peut-on concilier performance et bien-être au travail ?
Quiconque a déjà travaillé au sein d'une entreprise connaît l'importance d'améliorer la productivité. Alors que pendant longtemps cette quête productiviste visait principalement à accroître la rentabilité de l'entreprise, force est de reconnaître que l'amélioration de l'efficience organisationnelle est maintenant nécessaire pour simplement assurer la pérennité de l'organisation. Pendant longtemps, les travailleurs, comme les organisations, se sont crus à l'abri de la concurrence mondiale; rares étaient donc les mesures incitatives mises en place pour accroître la productivité. Or trois facteurs ont, entre autres, rapidement transformé cette réalité:
L'allongement des cycles de récession;
La mondialisation des marchés et le dynamisme de la concurrence internationale;
La pénurie de main-d'œuvre 

In [8]:
summaries = {idx: generate_summary(chapter) for idx, chapter in enumerate(chapters)}

Setting `pad_token_id` to `eos_token_id`:6 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:6 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:6 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:6 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:6 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:6 for open-end generation.


In [9]:
pp = PrettyPrinter(indent=2)
pp.pprint(summaries)
# TODO: Investigate how these summaries compare to the references
# TODO: Investigate why it's one sentence per summary
# TODO: Investigate "interpré- tation" in summary of chapter 4

{ 0: 'Le monde organisationnel est soumis à des exigences de productivité '
     "historiques, exigences qui demeurent, faut-il l'admettre, complexes à "
     'conjuguer avec une promotion et un soutien de la santé psychologique au '
     'travail.',
  1: 'Les entreprises qui se rendent compte que leur survie est en jeu se '
     "sentent de plus en plus contraintes d'accroître leur qualité de vie au "
     'travail.',
  2: 'Le coin du coach. La notion de stress reste mal comprise du grand '
     "public. Mais qu'est-ce exactement que le stress?",
  3: 'Le stress est un élément normal de notre vie quotidienne. Bien que ce ne '
     'soit pas toujours le cas, il est généralement associé à un état négatif '
     "ou à une expérience préjudiciable qu'il faut éliminer à tout prix",
  4: "La notion d'estime de soi est largement subjective, c'est-à-dire qu'il "
     "est la résultante d'un processus d'interpré- tation de l'environnement "
     "par l'individu.",
  5: "Le stress n'est pas en 